# 函数式编程

JavaScript“一等函数”
* 可以存储在变量中
* 可以存储在数组的元素中
* 可以存储在对象的字段中
* 可以根据需要来创建
* 可以被传递到其他函数中
* 可以被其他函数返回

高阶函数
* 接受一个或多个函数作为输入
* 输出一个函数

集合中心编程
> 用100个函数操作一个数据结构，比用10个函数操作10个数据结构要好。————Alan Perlis

Applicative函数：定义一个函数，参数包含函数、并调用传入的函数。

关键点：用较低级别的函数来逐步定义和使用离散功能。
一串函数链一个接一个地调用，每个函数将逐渐转变的结果传到后一个函数，最后得到解决方案。


In [4]:
function allOf() {
    return Array.prototype.reduceRight.call(arguments, function(truth, f) {
        return truth && f();
    },true);
}
function anyOf() {
    return Array.prototype.reduceRight.call(arguments, function(truth, f) {
        return truth || f();
    },false)
}

function T() {return true;}
function F() {return false;}

console.log(allOf());
console.log(allOf(T, T,T,F));
console.log(anyOf(T, T,T,F));

true
false
true


In [9]:
function mapcat(fn, collection) {
    return Array.prototype.concat.apply([], Array.prototype.map.call(collection, fn));
}

mapcat(item => [item, ','], [1,2,3,4])

[ 1, ',', 2, ',', 3, ',', 4, ',' ]

## 数据思考
深入思考应用的数据，以及每一层交接的格式。
SQL 表结构

In [22]:
var library = [
    {title: 'JavaScript', isbn: '131231231', ed: 1},
    {title: 'JavaScript', isbn: '131231232', ed: 2},
    {title: 'Python', isbn: '1313991231', ed: 1},
    {title: 'Java', isbn: '131231', ed: 6},
]

function pluck(obj, key) {return obj.map(item => item[key])}
function map(obj, fn) {return Array.prototype.map.call(obj, fn);}
function pick(obj, keys) {let result = {}; keys.forEach(key=> {result[key] = obj[key];}); return result;}
function omit(obj, keys) {let result = {}; Object.keys(obj).forEach(key => {if(keys.indexOf(key)===-1){result[key] = obj[key];}}); return result}
function construct(head, tail) {return [head].concat(tail);}

function project(table, keys) {
    return map(table, function(obj) {
       return pick.call(null, obj, keys); 
    });
}

console.log(project(library, ['title', 'isbn']));
console.log(pluck(project(library, ['title', 'isbn']), 'isbn'));

[ { title: 'JavaScript', isbn: '131231231' },
  { title: 'JavaScript', isbn: '131231232' },
  { title: 'Python', isbn: '1313991231' },
  { title: 'Java', isbn: '131231' } ]
[ '131231231', '131231232', '1313991231', '131231' ]


In [26]:
// SQL 的as功能 SELECT ed AS edition FROM library.
function rename (item, names) {
    let result = omit(item, Object.keys(names));
    Object.keys(names).forEach(key => {
        result[names[key]] = item[key];
    })
    return result;
}
function as (table, names) {
    return table.map(item => rename(item, names));
}
console.log(as(library, {ed: 'edition'}));

[ { title: 'JavaScript', isbn: '131231231', edition: 1 },
  { title: 'JavaScript', isbn: '131231232', edition: 2 },
  { title: 'Python', isbn: '1313991231', edition: 1 },
  { title: 'Java', isbn: '131231', edition: 6 } ]


## 柯里化 currying
> 柯里化 currying是把接受多个参数的函数变换成接受一个单一参数（最初函数的第一个参数）的函数，并且返回接受余下的参数而且返回结果的新函数的技术。这个技术由克里斯托弗·斯特雷奇以逻辑学家哈斯凯尔·加里命名的，尽管它是Moses Schönfinkel和戈特洛布·弗雷格发明的。

### 向左柯里化、还是向右柯里化


In [ ]:
function div (n, d) {
    return n/d;
}

function leftCurryDiv(n) {
    return function(d) {
        return n/d;
    }
}

function rightCurryDiv(d) {
    return function(n){
        return n/d;
    }
}

let divide10By = leftCurryDiv(10);
let divideBy10 = rightCurryDiv(10);

### 手动柯里化->自动柯里化

In [ ]:
function curry(fun) {
    return function(arg) {
        return fun(arg);
    }
}
// 感觉没什么用处。。

[11,11,11,11].map(parseInt)  //[ 11, NaN, 3, 4]
[11,11,11,11].map(curry(parseInt))  //[ 11, 11, 11, 11]

function curry2(fun) {
    return function(secondArg) {
        return function(firstArg) {
            return fun(firstArg, secondArg);
        }
    }
}

let divBy10 = curry2(div)(10);

### curry2, curry3, curryAll
部分应用，不是一个参数一个参数的用，而是一个两个或多个，类似bind

In [ ]:
function partial(fun, ...args) {
    return function (...oargs) {
        var allArgs = [...args, ...oargs];
        return fun.apply(fun, allArgs);
    }
}

### 前置条件、后置条件

给出函数能处理的数据，那么就能确保符合特定条件的结果。

## 纯度、不变性、改变政策
纯pure函数
* 函数结果只能从它的参数值来计算
* 不能依赖于被外部操作改变的数据
* 不能改变外部状态

提纯